In [4]:
!pip install "folium>=0.12" matplotlib mapclassify
# أو conda:
# conda install -c conda-forge "folium>=0.12" matplotlib mapclassify


   ---------------------------------------- 0.0/882.2 kB ? eta -:--:--
   --------------------------------------- 882.2/882.2 kB 13.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# -*- coding: utf-8 -*-
# ==========================================
#  VRP على شبكة طرق OSMnx (v2-ready)
#  - بناء شبكة من BBOX
#  - سرعات مخصصة + travel_time
#  - توليد نقاط "على الطريق" موزونة بطول الحواف
#  - اختيار ديبو ذكي يزيد الوصول الموجّه
#  - تقسيم Sweep + NN + 2-opt
#  - إحصائيات + خريطة تفاعلية (GeoPandas.explore)
# ==========================================

import math, random, functools, time, warnings
from pathlib import Path

import numpy as np
import networkx as nx
import osmnx as ox
from shapely.geometry import box, LineString, Point

# حاول استخدام GeoPandas + folium لعمل خريطة تفاعلية
try:
    import geopandas as gpd
    _HAS_GPD = True
except Exception:
    _HAS_GPD = False

try:
    import folium
    _HAS_FOLIUM = True
except Exception:
    _HAS_FOLIUM = False

# ------------------------------------------
# إعدادات عامة قابلة للتعديل
# ------------------------------------------
# صندوق الاهتمام (minx=west, miny=south, maxx=east, maxy=north) بوحدات lon/lat
WEST, SOUTH, EAST, NORTH = 36.71519, 34.00945, 36.74922, 34.03680
POLY = box(WEST, SOUTH, EAST, NORTH)

# مركبات، نقاط، مسافات، تحسين
V = 2                       # عدد المركبات
POINTS_N = 550              # عدد نقاط الخدمة المراد توليدها
MIN_SPACING_M = 80          # مسافة دنيا بين النقاط وقت التوليد
CLUSTER_RADIUS_M = 15       # دمج نقاط قريبة (اختياري)
USE_2OPT = True
MAX_2OPT_ITERS = 150

# تسريع/إبطاء أزمنة السفر (إشارة إلى إشارات/تقاطعات..)
CITY_TIME_FACTOR = 1.3

# أزمنة الخدمة العشوائية لكل نقطة (ثواني)
SERVICE_TIME_RANGE = (60, 120)

# تثبيت البذور لإعادة إنتاج النتائج
random.seed(42); np.random.seed(42)

# سرعات مخصصة حسب نوع الطريق (كم/س)
CUSTOM_SPEEDS = {
    "motorway":80, "motorway_link":60,
    "trunk":70, "trunk_link":60,
    "primary":60, "primary_link":50,
    "secondary":50, "secondary_link":45,
    "tertiary":40, "tertiary_link":35,
    "residential":30, "living_street":20,
    "service":20, "unclassified":35
}

# ------------------------------------------
# بناء شبكة الطرق مع travel_time (OSMnx v2)
# ------------------------------------------
print("📡 Loading road network BBOX...")
G = ox.graph_from_polygon(
    POLY,
    network_type="drive",
    retain_all=True,
    simplify=True,
    truncate_by_edge=True
)

# أضف السرعات ثم travel_time (v2: تحت osmnx.routing)
# https://osmnx.readthedocs.io/en/stable/internals-reference.html#osmnx.routing.add_edge_speeds
G = ox.routing.add_edge_speeds(G, hwy_speeds=CUSTOM_SPEEDS)
G = ox.routing.add_edge_travel_times(G)

# عامل مدينة للواقعية
for _, _, _, d in G.edges(keys=True, data=True):
    if "travel_time" in d:
        d["travel_time"] *= CITY_TIME_FACTOR

# تأكد من CRS = EPSG:4326 (lon/lat)
G = ox.project_graph(G, to_crs="EPSG:4326")

# خذ أكبر مكوّن ضعيف الاتصال (weakly) حتى لا نتقيّد بنقطة ديبو مسبقة
# https://osmnx.readthedocs.io/en/stable/internals-reference.html#osmnx.truncate.largest_component
G_cc = ox.truncate.largest_component(G, strongly=False)

Gud = G_cc.to_undirected(as_view=True)  # نسخة غير موجّهة للاحتياط
print(f"ℹ️ Nodes/Edges in largest CC: {G_cc.number_of_nodes()} / {G_cc.number_of_edges()}")

# تقدير سرعة fallback (م/ث) للاحتياط غير الموجّه
speeds_kph = []
for _, _, _, d in G_cc.edges(keys=True, data=True):
    sp = d.get("speed_kph", np.nan)
    if sp is not None:
        speeds_kph.append(sp)
fallback_mps = (np.nanmedian(speeds_kph) if len(speeds_kph) else 30.0) * 1000/3600.0

# ------------------------------------------
# توليد نقاط "على الطريق" موزونة بطول الحواف
# ------------------------------------------
# سنستخدم وزن الاحتمال = طول الحافة بالمتر (d["length"])
edges_for_sampling = []
cum = []
tot_m = 0.0
for u, v, k, d in G_cc.edges(keys=True, data=True):
    Lm = float(d.get("length", 0.0))  # متر
    if Lm <= 0:
        continue
    geom = d.get("geometry")
    if geom is None:
        # أنشئ خطًا مستقيمًا بين العقدتين (بالدرجات، لكن سنستخدم نسبة t فقط)
        geom = LineString([(G_cc.nodes[u]["x"], G_cc.nodes[u]["y"]),
                           (G_cc.nodes[v]["x"], G_cc.nodes[v]["y"])])
    edges_for_sampling.append((u, v, k, geom, Lm))
    tot_m += Lm
    cum.append(tot_m)

def _sample_edge_by_length():
    # اختيار حافة بوزن الطول (متر)
    r = random.random() * tot_m
    idx = next(i for i, c in enumerate(cum) if c >= r)
    return edges_for_sampling[idx]

def sample_point_on_edge():
    u, v, k, geom, Lm = _sample_edge_by_length()
    # اختر موضعًا كنسبة t على الهندسة (الوحدة درجات، لكن t نسبة)
    t = random.random()
    p = geom.interpolate(t * geom.length)  # param بوحدات degrees، t يضمن تناسب
    return (p.y, p.x)  # (lat, lon)

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2*R*math.asin(math.sqrt(a))

def generate_bin_points(n_points=POINTS_N, min_gap_m=MIN_SPACING_M, max_tries=10000):
    pts = []
    tries = 0
    while len(pts) < n_points and tries < max_tries:
        lat, lon = sample_point_on_edge()
        ok = True
        for (la, lo) in pts:
            if haversine_m(lat, lon, la, lo) < min_gap_m:
                ok = False
                break
        if ok:
            pts.append((lat, lon))
        tries += 1
    return pts

snapped_pts = generate_bin_points(POINTS_N, MIN_SPACING_M)
print(f"✅ Generated on-road points: {len(snapped_pts)}")

# اختياري: دمج نقاط متقاربة جدًا عبر BallTree لو متاح sklearn
try:
    from sklearn.neighbors import BallTree
    R_earth = 6371000.0
    pts_rad = np.radians(np.array(snapped_pts))
    tree = BallTree(pts_rad, metric='haversine')
    picked = np.zeros(len(snapped_pts), dtype=bool)
    clustered = []
    for i in range(len(snapped_pts)):
        if picked[i]: continue
        idxs = tree.query_radius(pts_rad[i:i+1], r=CLUSTER_RADIUS_M/R_earth)[0]
        clustered.append(snapped_pts[i])
        picked[idxs] = True
    snapped_pts = clustered
    print(f"🔧 After clustering within {CLUSTER_RADIUS_M} m: {len(snapped_pts)} points")
except Exception:
    pass

# حوّل كل نقطة إلى أقرب عقدة على الشبكة
# ملاحظة: nearest_nodes يأخذ X=lon, Y=lat (OSMnx v2 User Reference)
# https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.distance.nearest_nodes
bin_nodes = [ox.distance.nearest_nodes(G_cc, lon, lat) for (lat, lon) in snapped_pts]
# إزالة المكررات والتأكد أنها ضمن G_cc
bin_nodes = list(dict.fromkeys([n for n in bin_nodes if n in G_cc.nodes]))
print(f"✅ Points after mapping to unique reachable nodes: {len(bin_nodes)}")

# ------------------------------------------
# اختيار ديبو ذكي يزيد الوصول الموجّه
# ------------------------------------------
# مرشح مركزي: أقرب عقدة إلى مركز bbox للشبكة
xs = np.array([G_cc.nodes[n]["x"] for n in G_cc.nodes()])
ys = np.array([G_cc.nodes[n]["y"] for n in G_cc.nodes()])
cx, cy = float(xs.mean()), float(ys.mean())
center_node = ox.distance.nearest_nodes(G_cc, cx, cy)

# مرشح closeness على الجراف غير الموجه (سريع ومؤثر)
close = nx.closeness_centrality(Gud, distance="length")
topk = [n for n,_ in sorted(close.items(), key=lambda kv: kv[1], reverse=True)[:5]]

# 10 مصبات (bins) كمرشحين متنوعة مكانياً
rnd_bins = random.sample(bin_nodes, k=min(10, len(bin_nodes)))

candidates = list(dict.fromkeys([center_node] + topk + rnd_bins))

@functools.lru_cache(maxsize=200_000)
def _dir_time(u, v):
    if u == v: return 0.0
    try:
        return nx.shortest_path_length(G_cc, u, v, weight="travel_time")
    except nx.NetworkXNoPath:
        return float("inf")

def directed_reach_stats(src):
    cnt = 0
    tsum = 0.0
    for n in bin_nodes:
        tt = _dir_time(src, n)
        if math.isfinite(tt):
            cnt += 1
            tsum += tt
    return cnt, tsum

scores = [(n,)+directed_reach_stats(n) for n in candidates]
# أفضل: أكبر وصول موجّه ثم أقل مجموع زمن للوصول
best_node, best_reach, best_sum = sorted(scores, key=lambda t: (-t[1], t[2]))[0]

depot_node = best_node
DEPOT_LAT, DEPOT_LON = G_cc.nodes[depot_node]["y"], G_cc.nodes[depot_node]["x"]
print("🔎 Depot candidates (node:reach,sum_s):",
      ", ".join([f"{n}:{r},{int(s)}" for n, r, s in scores]))
print(f"✅ Picked depot node: {depot_node}  (directed reach {best_reach}/{len(bin_nodes)})")

# ------------------------------------------
# أدوات مساعدة للتوجيه
# ------------------------------------------
@functools.lru_cache(maxsize=100_000)
def travel_time_sec(u, v):
    """أقصر زمن بين عقدتين (ثواني) بمراعاة الاتجاه، مع احتياطي غير موجّه."""
    if u == v:
        return 0.0
    try:
        return nx.shortest_path_length(G_cc, u, v, weight="travel_time")
    except nx.NetworkXNoPath:
        try:
            # fallback غير موجّه بطول/سرعة تقريبية
            L = nx.shortest_path_length(Gud, u, v, weight="length")
            return L / fallback_mps
        except Exception:
            return float("inf")

@functools.lru_cache(maxsize=100_000)
def route_nodes(u, v):
    """العقد المكوّنة للمسار الأقصر (للرسم/الحساب)."""
    if u == v:
        return [u]
    # حاول موجّه أولاً
    try:
        return nx.shortest_path(G_cc, u, v, weight="travel_time")
    except nx.NetworkXNoPath:
        try:
            return nx.shortest_path(Gud, u, v, weight="length")
        except Exception:
            return []

def path_stats(u, v):
    """زمن/مسافة لمقطع واحد عبر route_to_gdf (v2)."""
    nodes = route_nodes(u, v)
    if len(nodes) < 2:
        return 0.0, 0.0
    try:
        # https://osmnx.readthedocs.io/en/stable/internals-reference.html#osmnx.routing.route_to_gdf
        gdf = ox.routing.route_to_gdf(G_cc, nodes, weight="travel_time")
        sec = float(gdf["travel_time"].sum()) if "travel_time" in gdf else 0.0
        meters = float(gdf["length"].sum()) if "length" in gdf else 0.0
        return sec, meters
    except Exception:
        # احتياطي: اجمع يدويًا من الحواف
        sec_total, m_total = 0.0, 0.0
        for a, b in zip(nodes[:-1], nodes[1:]):
            try:
                # خذ أقصر حافة بين a,b
                k, d = min(G_cc[a][b].items(), key=lambda kv: kv[1].get("length", float("inf")))
            except Exception:
                continue
            sec_total += float(d.get("travel_time", 0.0))
            m_total   += float(d.get("length", 0.0))
        return sec_total, m_total

# ------------------------------------------
# تقسيم بالنظام الزاوي (Sweep) + موازنة بسيطة
# ------------------------------------------
dep_y, dep_x = G_cc.nodes[depot_node]["y"], G_cc.nodes[depot_node]["x"]
def angle_of(n):
    y, x = G_cc.nodes[n]["y"], G_cc.nodes[n]["x"]
    return math.atan2(y - dep_y, x - dep_x)

ordered = sorted(bin_nodes, key=angle_of)
buckets = [ordered[i::V] for i in range(V)]

def rebalance_by_time(buckets, rounds=10):
    for _ in range(rounds):
        loads = [sum(travel_time_sec(depot_node, n) for n in b) for b in buckets]
        worst, best = int(np.argmax(loads)), int(np.argmin(loads))
        if not buckets[worst]:
            break
        moved = buckets[worst].pop()  # آخر عنصر (قرّب من طرف القطاع)
        buckets[best].append(moved)
    return buckets

buckets = rebalance_by_time(buckets, rounds=12)

# ------------------------------------------
# بناء المسارات (NN) + تحسين 2-opt
# ------------------------------------------
service_time_sec = {n: random.uniform(*SERVICE_TIME_RANGE) for n in bin_nodes}

def route_nn(start, targets):
    unvisited = set(targets)
    seq = [start]
    cur = start
    while unvisited:
        nxt = min(unvisited, key=lambda n: travel_time_sec(cur, n))
        if math.isinf(travel_time_sec(cur, nxt)):
            unvisited.remove(nxt)
            continue
        seq.append(nxt)
        cur = nxt
        unvisited.remove(nxt)
    seq.append(start)
    return seq

def two_opt(start, stops, iters=MAX_2OPT_ITERS):
    if len(stops) < 3:
        return stops
    best = stops[:]
    def cost(path):
        return sum(travel_time_sec(a, b) for a, b in zip([start]+path, path+[start]))
    best_cost = cost(best)
    for _ in range(iters):
        improved = False
        for i in range(len(best)-1):
            for j in range(i+2, len(best)):
                new = best[:i] + best[i:j][::-1] + best[j:]
                c = cost(new)
                if c < best_cost - 1e-6:
                    best, best_cost = new, c
                    improved = True
                    break
            if improved: break
        if not improved:
            break
    return best

vehicle_routes = []
for g in buckets:
    seq = route_nn(depot_node, g)
    stops = seq[1:-1]
    if USE_2OPT and len(stops) > 2:
        stops = two_opt(depot_node, stops, MAX_2OPT_ITERS)
    route = [depot_node] + stops + [depot_node]
    vehicle_routes.append(route)

# ------------------------------------------
# تحليل النتائج
# ------------------------------------------
reports = []
for i, seq in enumerate(vehicle_routes, 1):
    travel_sec, dist_m = 0.0, 0.0
    for a, b in zip(seq[:-1], seq[1:]):
        t, l = path_stats(a, b)
        travel_sec += t
        dist_m += l
    unique_stops = list(dict.fromkeys(seq[1:-1]))
    service_sec = sum(service_time_sec[n] for n in unique_stops)
    reports.append({
        "vehicle": i,
        "stops": len(unique_stops),
        "travel_min": travel_sec/60.0,
        "service_min": service_sec/60.0,
        "total_min": (travel_sec+service_sec)/60.0,
        "dist_km": dist_m/1000.0,
        "seq": seq
    })

# ------------------------------------------
# الطباعة النهائية
# ------------------------------------------
print("\n=== Results ===")
if reports:
    total_time = sum(r["total_min"] for r in reports)
    max_time = max(r["total_min"] for r in reports)
    for r in reports:
        print(f"🚚 Vehicle {r['vehicle']:>2}: {r['stops']:>3} stops | "
              f"travel={r['travel_min']:.1f}m | service={r['service_min']:.1f}m | "
              f"total={r['total_min']:.1f}m | dist={r['dist_km']:.2f} km")
    covered_nodes = set()
    for r in reports:
        covered_nodes.update(r["seq"][1:-1])
    print(f"\nCoverage: {len(covered_nodes)}/{len(bin_nodes)} points reached")
    print(f"Total time (all): {total_time/60.0:.1f} hours")
    print(f"Longest vehicle: {max_time/60.0:.1f} hours")
else:
    print("No routes produced.")

# ------------------------------------------
# رسم: خريطة تفاعلية (GeoPandas.explore) + بدائل
# ملاحظة: وحدة folium في OSMnx أُزيلت في v2 — استخدم .explore من GeoPandas
# https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html
# ------------------------------------------
out_map = "vrp_map.html"
try:
    if _HAS_GPD:
        edges_gdf = ox.graph_to_gdfs(G_cc, nodes=False)
        m = edges_gdf.explore(style_kwds={"opacity": 0.6, "weight": 2})

        if _HAS_FOLIUM:
            # ديبو
            folium.CircleMarker(location=[DEPOT_LAT, DEPOT_LON],
                                radius=6, color="green", fill=True).add_to(m)
            # نقاط الخدمة (باللون الأحمر)
            for n in bin_nodes:
                y, x = G_cc.nodes[n]["y"], G_cc.nodes[n]["x"]
                folium.CircleMarker(location=[y, x], radius=2, color="red", fill=True).add_to(m)
            # مسارات المركبات بألوان مختلفة
            colors = ["#10b981", "#3b82f6", "#f59e0b", "#ef4444", "#8b5cf6", "#14b8a6"]
            for i, r in enumerate(reports):
                col = colors[i % len(colors)]
                seq = r["seq"]
                for a, b in zip(seq[:-1], seq[1:]):
                    nodes = route_nodes(a, b)
                    if len(nodes) >= 2:
                        coords = [(G_cc.nodes[z]["y"], G_cc.nodes[z]["x"]) for z in nodes]
                        folium.PolyLine(coords, weight=3, opacity=0.9, color=col).add_to(m)
        m.save(out_map)
        print(f"🗺️ Saved interactive map: {out_map}")
    else:
        raise ImportError("GeoPandas not available")
except Exception as e:
    print("ℹ️ Interactive map not generated (", str(e), "). Falling back to static PNG...")
    import matplotlib.pyplot as plt
    fig, ax = ox.plot_graph(G_cc, show=False, close=False, node_size=0,
                            edge_color="#b8b8b8", edge_alpha=0.6, bgcolor="white", figsize=(7,7))
    # نقاط
    lats = [G_cc.nodes[n]["y"] for n in bin_nodes]
    lons = [G_cc.nodes[n]["x"] for n in bin_nodes]
    ax.scatter(lons, lats, s=10, c="red", zorder=5, label="Bins")
    ax.scatter(DEPOT_LON, DEPOT_LAT, s=60, c="lime", zorder=6, label="Depot")

    # مسارات
    colors = ["#10b981", "#3b82f6", "#f59e0b", "#ef4444", "#8b5cf6", "#14b8a6"]
    for i, r in enumerate(reports):
        col = colors[i % len(colors)]
        seq = r["seq"]
        for a, b in zip(seq[:-1], seq[1:]):
            nodes = route_nodes(a, b)
            if len(nodes) > 1:
                xs = [G_cc.nodes[z]["x"] for z in nodes]
                ys = [G_cc.nodes[z]["y"] for z in nodes]
                ax.plot(xs, ys, lw=2.5, alpha=0.9, color=col)
    plt.tight_layout()
    plt.savefig("optimized_vrp.png", dpi=200)
    plt.show()




📡 Loading road network BBOX...
ℹ️ Nodes/Edges in largest CC: 865 / 2330
✅ Generated on-road points: 550
🔧 After clustering within 15 m: 550 points
✅ Points after mapping to unique reachable nodes: 364
🔎 Depot candidates (node:reach,sum_s): 729765259:358,55584, 720711536:358,53286, 738481067:358,53147, 729716900:358,54110, 1351945438:358,53878, 1069557692:358,52391, 12422199136:358,76139, 729688549:358,83735, 3191439065:358,85412, 1069557685:358,71151, 12422199126:358,110199, 715884170:358,54723, 719653758:358,72432, 1069689543:358,74192, 4178558136:358,115695, 705902002:358,55221
✅ Picked depot node: 1069557692  (directed reach 358/364)

=== Results ===
🚚 Vehicle  1: 178 stops | travel=122.6m | service=269.7m | total=392.3m | dist=59.40 km
🚚 Vehicle  2: 186 stops | travel=142.6m | service=281.6m | total=424.2m | dist=67.32 km

Coverage: 364/364 points reached
Total time (all): 13.6 hours
Longest vehicle: 7.1 hours
🗺️ Saved interactive map: vrp_map.html
